In [1]:
import os
from dotenv import load_dotenv

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o-mini")
llm_response = llm.invoke("Tell me a joke")
llm_response

AIMessage(content='Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-10d7198e-0bfc-4681-9604-627e202b5d3c-0', usage_metadata={'input_tokens': 11, 'output_tokens': 17, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [3]:
from langchain_core.output_parsers import StrOutputParser

chain = llm | StrOutputParser()
chain.invoke("Tell me a joke")

'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'

### Structured Output

In [5]:
from pydantic import BaseModel, Field
from typing import List

class MobileReview(BaseModel):
    phone_model: str = Field(description="Model of the phone")
    rating: float = Field(description="Phone overall rating")
    pros: List[str] = Field(description="list of positives review")
    cons: List[str] = Field(description="list of negative review")
    summary: str = Field(description="overall summary of the review")


review_text = """
    Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
    colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
    stronger. Battery life's solid, lasts me all day no problem.
    Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
    Also, still getting used to the new button layout, keep hitting Bixby by mistake.
    Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
    being perfect. If you're due for an upgrade, definitely worth checking out!
    """

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
output

MobileReview(phone_model='Galaxy S21', rating=4.0, pros=['Gorgeous screen', 'Vibrant colors', 'Insane camera performance, especially at night', 'Solid battery life, lasts all day'], cons=['Pretty pricey', 'No charger included', 'New button layout takes getting used to, accidental Bixby activation'], summary='Overall, a solid 4 out of 5. Great phone, but a few annoying quirks keep it from being perfect. Definitely worth checking out for an upgrade!')

### Chat template

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
chain = prompt | llm | StrOutputParser()

chain.invoke({"topic": "programmer"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!'

### LLM messages

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage

systemsg = SystemMessage("You are a helpful financial advisor giving detailed insight about the stock. For any query outside finances, you just output you're not made for this task.")
humanmsg = HumanMessage("Give me the fundamental analysis of Nvidia stock.")

chain = llm | StrOutputParser()
res = chain.invoke([systemsg, humanmsg])
res

"Nvidia Corporation (NVDA) is a leading technology company primarily known for its graphics processing units (GPUs) and increasingly for its advancements in artificial intelligence (AI) and data center solutions. Here’s a fundamental analysis of Nvidia stock based on various financial metrics and qualitative factors.\n\n### 1. **Revenue Growth**\nNvidia has demonstrated robust revenue growth over the past few years, driven by strong demand for gaming, data centers, AI, and automotive applications. With the increasing reliance on AI and machine learning, Nvidia's role as a provider of GPUs and AI computing solutions positions it well for future growth.\n\n### 2. **Earnings Performance**\nNvidia has shown consistent growth in earnings per share (EPS). The company has benefited from high-margin products and has managed its costs effectively, leading to strong net income growth. Investors often look at the EPS growth rate as an indicator of the company’s profitability trajectory.\n\n### 3.

In [9]:
humanmsg = HumanMessage("Give me the detailed analysis of BRICS vs NATO history.")

chain = llm | StrOutputParser()
res = chain.invoke([systemsg, humanmsg])
res

"I'm not made for this task."

In [10]:
template = ChatPromptTemplate(
    [
        ("system", "You give historical context of anything. And you don't do any other tasks."),
        ("human", "Tell me about {topic}")
    ]
)

chain = template | llm | StrOutputParser()
res = chain.invoke({"topic": "Bitcoin"})
res

'Bitcoin, created in 2009 by an anonymous entity known as Satoshi Nakamoto, is the first decentralized digital currency. Its inception followed the 2008 financial crisis, which exposed vulnerabilities in the traditional banking system and fueled interest in alternative financial systems.\n\nThe Bitcoin white paper, titled "Bitcoin: A Peer-to-Peer Electronic Cash System," outlined a method for using a peer-to-peer network to create a system of electronic cash that did not rely on a central authority. This was achieved through blockchain technology, a distributed ledger that records all transactions across a network of computers, ensuring transparency and security.\n\nThe first block of the Bitcoin blockchain, known as the "genesis block," was mined by Nakamoto in January 2009. Bitcoin gained initial traction among cryptography enthusiasts and was primarily used for online transactions. Its value began to rise significantly in 2011, leading to increased media attention and wider adoption

In [11]:
chain = template | llm | StrOutputParser()
res = chain.invoke({"topic": "Future of Internet"})
res

'I can provide historical context on the development of the internet and its evolution over time, but I cannot offer predictions about its future. \n\nThe internet originated from research projects in the 1960s, notably ARPANET, which aimed to create a robust, fault-tolerant communication system. By the 1980s, the adoption of TCP/IP protocols allowed different networks to interconnect, leading to the expansion of what we now know as the internet.\n\nIn the 1990s, the commercialization of the internet began, with the introduction of the World Wide Web by Tim Berners-Lee, which made it accessible to the general public. This era saw an explosion of websites, email usage, and the birth of e-commerce.\n\nThe 2000s brought the rise of social media, mobile internet access, and the proliferation of broadband connections, fundamentally changing how people interact and access information. The development of cloud computing also transformed the landscape, enabling vast amounts of data to be store

### Creating the retriever

In [13]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

pdf_loader = PyPDFLoader("Essay.pdf")
pdf = pdf_loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(pdf)

splits

[Document(metadata={'source': 'Essay.pdf', 'page': 0}, page_content='Why one should write?  \n \nWhen we look into the lives of some of the most successful leaders, ancient philosophers, innovators, \nphilanthropists from ancient times to modern times, there is one quality that distinguishes them from \nthe rest: Clarity of thought!  It is this clarity of thought that helps them build great organizations, \nlead people, bring new innova tions and inspire generations.  \nSo the ultimate question revolves around how they became so clear with their thoughts? Steve Jobs'),
 Document(metadata={'source': 'Essay.pdf', 'page': 0}, page_content='was very clear about what he wanted his produ cts to be and what kind of people he wanted in his \norganization. Chanakya was very clear with his vision of a united India and its leadership. Lee Kuan \nYew, a brilliant statesman, was clear with his vision of a modern and economic powerhouse island \nstate, Singapore. Modi is clear with his vision of Ind

In [14]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embd = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

embd

d:\DeepLearningMastery\LLM\Langchain\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': False}, multi_process=False, show_progress=False)

In [15]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embd,
    collection_name="my_collection",
    persist_directory="./chroma_db"
)

query = "people who still writes"
search_results = vectorstore.similarity_search(query, k=3)

print(search_results)

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
retriever.invoke("Name some people who still writes")

[Document(metadata={'page': 0, 'source': 'Essay.pdf'}, page_content='Sam Altman still writes, Ben Horowitz still writes and all those shaping the future still writes. Writing  \nsimply amplifies your thoughts!  \nReference 1/ https://fortune.com/2023/08/24/peter -thiel-student-newspaper -stanford -review/  \nIf you write, you will read, if you read, you will gain knowledge, if you gain knowledge, you will \nunderstand the world, and if you understa nd the world, you will have solutions to its problems and,'), Document(metadata={'page': 0, 'source': 'Essay.pdf'}, page_content='own! \nNot just critica l thinking and clear thoughts, writing will help you connect to the world, communicate \nbetter, make you smarter, disciplined, have ocean of knowledge and a stronger memory.  \nReference 3/ \nhttps://x.com/pritika_mehta/status/1790799256783393204?t=qosur91exNY43VLf0qzOdg& s=08  \n \n@silver_cule on X'), Document(metadata={'page': 0, 'source': 'Essay.pdf'}, page_content='executors. This is 

[Document(metadata={'page': 0, 'source': 'Essay.pdf'}, page_content='Sam Altman still writes, Ben Horowitz still writes and all those shaping the future still writes. Writing  \nsimply amplifies your thoughts!  \nReference 1/ https://fortune.com/2023/08/24/peter -thiel-student-newspaper -stanford -review/  \nIf you write, you will read, if you read, you will gain knowledge, if you gain knowledge, you will \nunderstand the world, and if you understa nd the world, you will have solutions to its problems and,'),
 Document(metadata={'page': 0, 'source': 'Essay.pdf'}, page_content='own! \nNot just critica l thinking and clear thoughts, writing will help you connect to the world, communicate \nbetter, make you smarter, disciplined, have ocean of knowledge and a stronger memory.  \nReference 3/ \nhttps://x.com/pritika_mehta/status/1790799256783393204?t=qosur91exNY43VLf0qzOdg& s=08  \n \n@silver_cule on X'),
 Document(metadata={'page': 0, 'source': 'Essay.pdf'}, page_content='executors. This i

### RAG Chain

In [16]:
template = """
Answer the question based on the following context:
{context}

Question: {question}

Answer:
"""

chat_temp = ChatPromptTemplate.from_template(template)

In [18]:
from langchain.schema.runnable import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | chat_temp
)

rag_chain.invoke("Name some people who still writes")

ChatPromptValue(messages=[HumanMessage(content="\nAnswer the question based on the following context:\n[Document(metadata={'page': 0, 'source': 'Essay.pdf'}, page_content='Sam Altman still writes, Ben Horowitz still writes and all those shaping the future still writes. Writing  \\nsimply amplifies your thoughts!  \\nReference 1/ https://fortune.com/2023/08/24/peter -thiel-student-newspaper -stanford -review/  \\nIf you write, you will read, if you read, you will gain knowledge, if you gain knowledge, you will \\nunderstand the world, and if you understa nd the world, you will have solutions to its problems and,'), Document(metadata={'page': 0, 'source': 'Essay.pdf'}, page_content='own! \\nNot just critica l thinking and clear thoughts, writing will help you connect to the world, communicate \\nbetter, make you smarter, disciplined, have ocean of knowledge and a stronger memory.  \\nReference 3/ \\nhttps://x.com/pritika_mehta/status/1790799256783393204?t=qosur91exNY43VLf0qzOdg& s=08  \\

In [19]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | chat_temp
    | llm
    | StrOutputParser()
)

question = "Name some people who still writes"
rag_chain.invoke(question)

'Some people who still write include Sam Altman and Ben Horowitz.'